## Pandas et scikit-learn

## Données du titanic (challenge Kaggle)

Camille Marini  
Repris d'un notebook d'Alexandre Gramfort donné pour un workshop Python sur ["Predictive Modeling with scikit-learn and pandas"](https://github.com/camillemarini/sklearn_pandas_intro).

Pour cette session, nous allons utiliser [pandas](http://pandas.pydata.org/), une librairie qui permet de manier facilement des données, et [sklearn](http://scikit-learn.org/stable/), une librairie de machine learning.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Chargement des données dans un DataFrame pandas 

Nous allons utiliser le jeu de données du challenge Kaggle sur la prédiction de la survie à bord du Titanic:
https://www.kaggle.com/c/titanic-gettingStarted

Pour télécharger les données: 

In [ ]:
!wget https://www.dropbox.com/s/6wu5fwj1i6cju2i/titanic_train.csv?dl=0

Pour charger le fichier csv dans un DataFrame pandas:

In [ ]:
# data = pd.read_csv('https://www.dropbox.com/s/6wu5fwj1i6cju2i/titanic_train.csv?dl=0')
data = pd.read_csv('titanic_train.csv')

Les DataFrames pandas sont affichés dans des tableaux html dans les jupyter notebook. Regardons les 5 premières lignes: 

In [ ]:
data.head(5)

In [ ]:
list(data.columns)

Les différentes données sont expliquées sur le site du challenge:

https://www.kaggle.com/c/titanic-gettingStarted/data

In [ ]:
data.count()

Le DataFrame a 891 lignes. Il manque des données pour certains passagers. 

In [ ]:
data.shape

On peut convertir un DataFrame en un numpy array avec:

In [ ]:
data.values

Le problème est qu'on ne peut pas directement donner ce DataFrame comme entrée d'un modèle scikit-learn, car:

* la variable cible (`Survived`) est avec les variables d'entrée  
* certains attributs comme les ids (`PassengerId`) n'ont aucune valeur prédictive.  
* Les données sont hétérogènes: string et des nombres.  
* certaines données sont manquantes (`nan`: "not a number")  

On va utiliser pandas pour préparer ces données.

## Prédire la survie

Le but du challenge est de prévoir si un passager a survécu à partir d'autres attributs connus. Commençons par regarder la colonne `Survived`:

In [ ]:
survived_column = data['Survived']
survived_column.dtype

`data.Survived` est une instance de la classe `Series` de pandas avec un dtype integer:

In [ ]:
type(survived_column)

`data` est une instance de la classe `DataFrame` de pandas:

In [ ]:
type(data)

Les instances de `Series` correspondent à des données 1D homogènes, alors que les instances de `DataFrame` sont des collections hétérogènres de colonnes de même longueur. 

Le DataFrame original peut être aggrégé en comptant les lignes pour chaque valeur possible de la variable `Survived`:

In [ ]:
data.groupby('Survived').count()

In [ ]:
data.groupby('Survived')

In [ ]:
np.mean(survived_column == 0)

Dans ces données, 62% des passagers ont péri (68% sur l'ensemble des passagers). On peut choisir comme modèle de référence un modèle qui prédirait constamment la non survie du passager. Il aurait une accuracy de 62% (ce qui est plus grand que le hasard).

On peut convertir les instances `Series` de pandas en un 1D numpy arrays en utilisant l'attribut `values`:

In [ ]:
target = survived_column.values

In [ ]:
type(target)

In [ ]:
target.dtype

In [ ]:
target[:5]

## Entraîner un modèle prédictif sur des features numériques


Les estimateurs `sklearn` acceptent des features numériques passées comme un numpy array. On ne peut donc pas passer le DataFrame brut. 

On commence simplement en construisant un modèle qui utilise seulement les features numériques données telles quelles: `data.Fare`, `data.Pclass` et `data.Age`.

In [ ]:
numerical_features = data.get(['Fare', 'Pclass', 'Age'])
numerical_features.head(5)

Malheureusement, il manque l'âge de certains passagers:

In [ ]:
numerical_features.count()

On peut utiliser la méthode `fillna` de pandas pour remplacer les `nan` par l'âge médian des passagers:

In [ ]:
median_features = numerical_features.dropna().median()
median_features

In [ ]:
imputed_features = numerical_features.fillna(median_features)
imputed_features.count()

In [ ]:
imputed_features.head(5)

Maintenant que le DataFrame est propre, on peut le convertir en un numpy array:

In [ ]:
features_array = imputed_features.values
features_array

In [ ]:
features_array.dtype

Prenons 80% des données pour l'entraînement et gardons 20% pour calculer le score de généralisation:

In [ ]:
from sklearn.model_selection import train_test_split

features_train, features_test, target_train, target_test = train_test_split(
    features_array, target, test_size=0.20, random_state=0)

In [ ]:
features_train.shape

In [ ]:
features_test.shape

In [ ]:
target_train.shape

In [ ]:
target_test.shape

Commençons avec un modèle simple de sklearn: [sklearn.linear_model.LogisticRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)

**Question**:
* Calculer les prédictions du modèle  
* Calculer l'accuracy de notre modèle. Est ce mieux que le modèle de référence qui prédit toujours la non survie?

In [ ]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(C=1.)
logreg.fit(features_train, target_train)

In [ ]:
target_predicted = logreg.predict(features_test)

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(target_test, target_predicted)

Notre premier modèle a une accuracy de 73%. C'est mieux que notre modèle de référence qui prédit toujours la non survie!

In [ ]:
logreg.score(features_test, target_test)

## Evaluation du modèle et interprétation

### Interpréter les poids du modèle linéaire

L'attribut `coef_` d'un modèle linéaire entraîné (tel que `LogisticRegression`) contient les poids de chaque feature:

In [ ]:
feature_names = numerical_features.columns
feature_names

In [ ]:
logreg.coef_

In [ ]:
x = np.arange(len(feature_names))
plt.bar(x, logreg.coef_.ravel())
_ = plt.xticks(x + 0.5, feature_names, rotation=30)

Dans notrre modèle, le `Fare` a une influence positive sur la survie, tandis que la `Pclass` et l'`Age` ont une influence négative. 

Les cabines de premières classes étaient plus proches des canots de sauvetage et que les femmes et enfants étaient évacués en priorité. Notre modèle semble capturer ces données historiques! On verra plus tard si le sexe des passagers est une information utilse pour augmenter les performances de notre modèle.

### Méthodes d'évaluation alternatives

On peut utiliser la matrice de confusion pour obtenir les détails des faux positifs et faux négatifs. 

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(target_test, target_predicted)
print(cm)

Les vraies étiquettes correspondent aux lignes et les prédites aux colonnes.

**Question**:
* faire un plot de la matrice de confusion  

In [ ]:
def plot_confusion(cm):
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.binary)
    plt.title('Confusion matrix')
    plt.set_cmap('Blues')
    plt.colorbar()

    target_names = ['not survived', 'survived']

    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=60)
    plt.yticks(tick_marks, target_names)
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    # Convenience function to adjust plot parameters for a clear layout.
    plt.tight_layout()
    
plot_confusion(cm)

On peut normaliser le nombre de prédiction en divisant par le nombre total de vrais `survived` et `not survived` pour calculer les taux de faux et vrais postifs pour la survie.

In [ ]:
print(cm)
cm.sum(axis=1)

In [ ]:
print(cm.astype(np.float64) / cm.sum(axis=1))

On observe que comme le jeu de données n'est pas balancé (peu de cas de survie par rapport à la non survie), le score d'accuracy n'est pas très informatif: il est assez bon, alors qu'on arrive très mal à prédire les cas de survie. 

On peut utiliser d'autres métriques pour évaluer la performance des modèles pour les jeux de données non balancés: precision, recall et le f1-score.

**Question**:
* Calculer ces métriques en utilisant les fonctions de scikit-learn.

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(target_test, target_predicted,
                            target_names=['not survived', 'survived']))

La régression logistique est un modèle probabiliste: il ne prédit pas qu'un output binaire (survived or not), mais il estime une probabilité que l'on peut obtenir avec la méthode `predict_proba`:

In [ ]:
target_predicted_proba = logreg.predict_proba(features_test)
target_predicted_proba[:5]

Par défault, la seuil de décision est à 0.5. Si on varie ce seuil de 0 à 1, on peut générer une famille de classifieurs binaires qui correspondent à différents compromis entre les taux de faux positifs et faux négatifs.

On peut résumer les performances de cette famille en plottant la courbe ROC et en calculant l'AUC (Area Under the Curve).

**Question**:
* en utilisant `sklearn.metrics.roc_curve` et `sklearn.metrics.auc`, dessiner cette courbe et afficher la valeur de l'AUC

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

def plot_roc_curve(target_test, target_predicted_proba):
    fpr, tpr, thresholds = roc_curve(target_test, target_predicted_proba[:, 1])
    
    roc_auc = auc(fpr, tpr)
    # Plot ROC curve
    plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
    plt.plot([0, 1], [0, 1], 'k--')  # random predictions curve
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('False Positive Rate or (1 - Specifity)')
    plt.ylabel('True Positive Rate or or recall or (Sensitivity)')
    plt.title('Receiver Operating Characteristic')
    plt.legend(loc="lower right")

In [ ]:
plot_roc_curve(target_test, target_predicted_proba)

Ici, l'AUC vaut 0.756, ce qui est similaire à l'accuracy de notre modèle (0.732).   
L'AUC d'un modèle random vaut 0.5, tandis que l'accuracy est influencé par le fait que le jeu de données n'est pas bien balancé. L'AUC peut être vue comme une façon de calibrer l'accuracy d'un modèle en prenant en compte le fait que le jeu de données n'est pas bien balancé.

### Validation croisée

Il est important de faire de la validation croisée pour évaluer notre modèle. 

**Question**:

- Calculer les scores "cross-validés" pour différentes métriques ('AUC', 'precision', 'recall', 'f1', 'accuracy'...).

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(logreg, features_array, target, cv=5)
scores

In [ ]:
scores.min(), scores.mean(), scores.max()

In [ ]:
scores = cross_val_score(logreg, features_array, target, cv=5,
                         scoring='roc_auc')
scores.min(), scores.mean(), scores.max()

In [ ]:
for k in range(3, 11):
    %time scores = cross_val_score(logreg, features_array, target, cv=k, scoring='roc_auc')
    print(scores.min(), scores.mean(), scores.max())

## Entraîner un modèle prédictif sur des features plus complexes

On va maintenant essayer de construire des modèles plus riches en incluant plus de features. 

Les variables catégorielles, telles que `data.Embarked` ou `data.Sex` peuvent être converties comme des booléens, appelés "dummy variables" ou "one-hot-encoded features":

In [ ]:
pd.get_dummies(data.Sex, prefix='Sex').head(5)

In [ ]:
pd.get_dummies(data.Embarked, prefix='Embarked').head(5)

**Question**: 
* Combiner ces nouvelles variables numériques avec les précédentes features dans un pandas DataFrame (appelé `rich_features`) en utilisant `pandas.concat`  

In [ ]:
rich_features = pd.concat([data.get(['Fare', 'Pclass', 'Age']),
                           pd.get_dummies(data.Sex, prefix='Sex'),
                           pd.get_dummies(data.Embarked, prefix='Embarked')],
                          axis=1)
rich_features.head(5)

Par construction, la nouvelle feature `Sex_male` est redondante avec  `Sex_female`. On peut l'enlever:

In [ ]:
rich_features_no_male = rich_features.drop('Sex_male', 1)
rich_features_no_male.head(5)

N'oublions pas d'imputer la valeur d'âge médian pour les passagers sans information d'âge:

In [ ]:
rich_features_no_male.count()

In [ ]:
rich_features_final = rich_features_no_male.fillna(rich_features_no_male.dropna().median())
rich_features_final.count()

**Question**:
* Calculer les scores "cross-validés" d'un modèle de régression logistique utilisant ces nouvelles features.

In [ ]:
%%time

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

logreg = LogisticRegression(C=1.)
scores = cross_val_score(logreg, rich_features_final, target, cv=5, scoring='accuracy')
print("Logistic Regression CV scores:")
print("min: {:.3f}, mean: {:.3f}, max: {:.3f}".format(
    scores.min(), scores.mean(), scores.max()))

**Question**:

* Afficher les poids des features de ce nouveau modèle de régression logistique. 

In [ ]:
logreg_new = LogisticRegression(C=1).fit(rich_features_final, target)            
                                                                                 
feature_names = rich_features_final.columns.values                               
x = np.arange(len(feature_names))                                                
plt.bar(x, logreg_new.coef_.ravel())                                             
_ = plt.xticks(x + 0.5, feature_names, rotation=30)                              
                                                                                 
# Rich young women like Kate Winslet tend to survive the Titanic better          
# than poor men like Leonardo.  

## Utiliser les pipelines

Quand on a rempli les valeurs manquantes par les valeurs médianes (imputation) avant de calculer les ensembles de train et de test, on utilise des données de test, ce qui est tricher...

Pour éviter cela, on devrait calculer les valeurs médianes seulement sur les données d'éntraînement et imputer ces valeurs à la fois sur les données d'entraînement et de test.

Pour cela, on peut préparer les features comme précédemment mais sans l'imputation, puis on utilise `sklearn.preprocessing.Imputer` pour calculer les valeurs médianes sur l'ensemble d'entraînement et les imputer aux valeurs manquantes sur l'ensemble d'entraînement et de test. On utilise enfin un `sklearn.pipeline.Pipeline` pour mettre tout ça ensemble.

In [ ]:
features = pd.concat([data.get(['Fare', 'Age']),
                      pd.get_dummies(data.Sex, prefix='Sex'),
                      pd.get_dummies(data.Pclass, prefix='Pclass'),
                      pd.get_dummies(data.Embarked, prefix='Embarked')],
                     axis=1)
features = features.drop('Sex_male', 1)
features.head(10)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features.values, target, random_state=0)

In [ ]:
from sklearn.preprocessing import Imputer

imputer = Imputer(strategy='median', missing_values="NaN")

imputer.fit(X_train)

Les valeurs médianes sont enregistrées dans l'attribut `statistics_`.

In [ ]:
imputer.statistics_

L'imputation se fait en appellant la méthode `transform`:

In [ ]:
X_train_imputed = imputer.transform(X_train)
X_test_imputed = imputer.transform(X_test)

On utilise maintenant un pipeline pour combiner l'imputation et le classifieur: 

In [ ]:
from sklearn.pipeline import Pipeline

imputer = Imputer(strategy='median', missing_values="NaN")

classifier = LogisticRegression(C=1.)

pipeline = Pipeline([
    ('imp', imputer),
    ('clf', classifier),
])

scores = cross_val_score(pipeline, features.values, target, cv=5, n_jobs=4,
                         scoring='accuracy', )
print(scores.min(), scores.mean(), scores.max())

### Crédits

Merci à:
* Alexandre Gramfort pour ce notebook  
* Kaggle pour la mise en place de ce challenge Titanic  
* Ce blog post de Philippe Adjiman dont s'est inspiré A. Gramfort:
http://www.philippeadjiman.com/blog/2013/09/12/a-data-science-exploration-from-the-titanic-in-r/